In [88]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.llms.openai import OpenAI
from langchain_openai import ChatOpenAI
import pandas as pd

In [7]:
import os
tavily_api_key = os.environ.get('TAVILY_API_KEY')

In [46]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
annotations = Chroma(persist_directory="db/kora-annotations", embedding_function=embedding_function)
opinions = Chroma(persist_directory="db/kora-opinions", embedding_function=embedding_function)

In [137]:
llm = ChatOpenAI(temperature=0, 
                 #openai_api_key=user_secrets.get_secret("OPENAI_API_KEY"), 
                 model_name='gpt-4-0125-preview')

In [138]:
annotation_retriever = MultiQueryRetriever.from_llm(
    retriever=annotations.as_retriever(), llm=llm
)
def get_metadata_and_content(documents):
  """
  Gets the metadata and page content of a list of documents.

  Args:
    documents: A list of Document objects.

  Returns:
    A string with the page_content and metadata Key: value pairs concatenated
    by a newline, and each item in the list concatenated by a double newline.
  """

  output = ""
  for document in documents:
    output += document.page_content + "\n"
    for key, value in document.metadata.items():
      output += "{}: {}\n".format(key, value)
    output += "\n\n"

  return output

@tool
def find_case_law_annotations(query: str) -> str:
    """Always search for annotations on relevant appelate court cases, which are binding for interpreting the law in Kentucky."""
    citations = annotation_retriever.invoke(query) 
    return get_metadata_and_content(citations)

In [48]:
opinion_retriever = opinions.as_retriever();

@tool
def find_attorney_general_annotations(query: str) -> str:
    """Perform a semantic search for relevant attorney general opinions. Only use this tool if you still need more context after looking for annotations"""
    citations = opinion_retriever.invoke(query) 
    return get_metadata_and_content(citations)

In [49]:
print(find_case_law_annotations("Complaints against a doctor"))

The attempt by the Board of Medical Licensure to categorize complaints against physicians as formal public complaints and private individual complaints has no bearing on whether such complaints must be released under the Open Records Act. Inasmuch as final actions stem from the complaints, they must be incorporated as part of the final determination and are therefore not exempt under subdivisions (1)(g) or (1)(h) of this section. Kentucky State Bd. of Medical Licensure v. Courier-Journal & Louisville Times Co., 663 S.W.2d 953, 1983 Ky. App. LEXIS 327 (Ky. Ct. App. 1983).
Category: Preliminary Materials
Statute Number: 61.878
Type: Annotation


Once final disciplinary action is taken by the Board of Medical Licensure, any complaint, report, memorandum, or letter made part of the record in such action is not within the exceptions to the Open Records Act found in subdivisions (1)(g) and (1)(h) of this section and must therefore be made available to the public. Kentucky State Bd. of Medica

In [53]:
# The Statute Numbers look like floats to the default importer, so we'll define the shape of the data.
statutes_dict = {
    'Chapter Number': 'object', 
    'Statute Number': 'object', 
    'Chapter Title': 'object', 
    'Section': 'object', 
    'Statute Title': 'object',
    'Statute Body': 'object',
    'History': 'object',
}

# Read the CSV file of all statutes in KRS Title VIII
df_statutes = pd.read_csv('krs/gov.ky.krs.title.08.statutes.csv', dtype=statutes_dict)
print(df_statutes.head())

  Chapter Number Statute Number  \
0             61         61.010   
1             61         61.012   
2             61         61.015   
3             61         61.020   
4             61         61.030   

                                       Chapter Title             Section  \
0  General Provisions as to Offices and Officers ...  General Provisions   
1  General Provisions as to Offices and Officers ...  General Provisions   
2  General Provisions as to Offices and Officers ...  General Provisions   
3  General Provisions as to Offices and Officers ...  General Provisions   
4  General Provisions as to Offices and Officers ...  General Provisions   

                                       Statute Title  \
0  Office not to be sold — Penalty — Contracts to...   
1  Qualification for appointive office or positio...   
2  Residence requirements for office or employmen...   
3      Commissions, which officers required to have.   
4   Discharge of duties, when officer to enter upon.

In [57]:
df_exceptions = pd.read_csv('exceptions.csv')
print(df_exceptions.head())

  Statute Number                                               Body
0   61.878(1)(a)  Public records containing information of a per...
1   61.878(1)(b)  Records confidentially disclosed to an agency ...
2   61.878(1)(c)  1. Records confidentially disclosed to an agen...
3   61.878(1)(d)  Public records pertaining to a prospective loc...
4   61.878(1)(e)  Public records which are developed by an agenc...


In [130]:
find_case_law_annotations, find_attorney_general_annotations
@tool
def lookup_exception_text(query: str) -> str:
    """Get the official text for a specific exception to the open records law by granular number, e.g. 61.878(1)(b). When you only need to check certain exceptions, this is more targeted than the full statute. Be sure to look up case law and AG opinions for context in interpretation"""
    result = df_exceptions[(df_exceptions['Statute Number'] == query)]
    if result.empty:
        return ''
    else:
        return 'Exception ' + query + ': ' + result.iloc[0].to_dict()['Body'] + """

There are some caveats to any exemption from 61.878:
61.878(2) No exemption in this section shall be construed to prohibit disclosure of statistical information not descriptive of any readily identifiable person.
61.878(3) No exemption in this section shall be construed to deny, abridge, or impede the right of a public agency employee, including university employees, an applicant for employment, or an eligible on a register to inspect and to copy any record including preliminary and other supporting documentation that relates to him. The records shall include, but not be limited to, work plans, job performance, demotions, evaluations, promotions, compensation, classification, reallocation, transfers, lay-offs, disciplinary actions, examination scores, and preliminary and other supporting documentation. A public agency employee, including university employees, applicant, or eligible shall not have the right to inspect or to copy any examination or any documents relating to ongoing criminal or administrative investigations by an agency.
61.878(4) If any public record contains material which is not excepted under this section, the public agency shall separate the excepted and make the nonexcepted material available for examination.
61.878(5) The provisions of this section shall in no way prohibit or limit the exchange of public records or the sharing of information between public agencies when the exchange is serving a legitimate governmental need or is necessary in the performance of a legitimate government function.
61.878(6) When material is made available pursuant to a request under subsection (1)(q) of this section, the public agency shall not be required to make a copy of the recording except as provided in KRS 61.169, and the requesting parties shall not be limited in the number of times they may view the material.
    """
print(lookup_exception_text("61.878(1)(b)"))

Exception 61.878(1)(b): Records confidentially disclosed to an agency and compiled and maintained for scientific research. This exemption shall not, however, apply to records the disclosure or publication of which is directed by another statute;

There are some caveats to any exemption from 61.878:
61.878(2) No exemption in this section shall be construed to prohibit disclosure of statistical information not descriptive of any readily identifiable person.
61.878(3) No exemption in this section shall be construed to deny, abridge, or impede the right of a public agency employee, including university employees, an applicant for employment, or an eligible on a register to inspect and to copy any record including preliminary and other supporting documentation that relates to him. The records shall include, but not be limited to, work plans, job performance, demotions, evaluations, promotions, compensation, classification, reallocation, transfers, lay-offs, disciplinary actions, examination

In [84]:
@tool
def lookup_statute_text(query: str) -> str:
    """Get the official text for a specific statute in the Kentucky Open Records Act by statute number"""
    result = df_statutes[(df_statutes['Statute Number'] == query)]
    if result.empty:
        return ''
    else:
        result = result.iloc[0].to_dict()
        return """KRS {}: {}
{}

In Chapter {}: {}
Section: {}
""".format(result['Statute Number'], result['Statute Title'], result['Statute Body'], result['Chapter Number'], result['Chapter Title'], result['Section'])
print(lookup_statute_text("61.870"))

KRS 61.870: Definitions for KRS 61.870 to 61.884.
As used in KRS 61.870 to 61.884, unless the context requires otherwise:
	(1) “Public agency” means:
		(a) Every state or local government officer;
		(b) Every state or local government department, division, bureau, board, commission, and authority;
		(c) Every state or local legislative board, commission, committee, and officer;
		(d) Every county and city governing body, council, school district board, special district board, and municipal corporation;
		(e) Every state or local court or judicial agency;
		(f) Every state or local government agency, including the policy-making board of an institution of education, created by or pursuant to state or local statute, executive order, ordinance, resolution, or other legislative act;
		(g) Any body created by state or local authority in any branch of government;
		(h) Any body which, within any fiscal year, derives at least twenty-five percent (25%) of its funds expended by it in the Commonw

In [131]:
tools = [find_case_law_annotations, find_attorney_general_annotations, lookup_exception_text, lookup_statute_text]

In [132]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("jscotthorn/openai-ky-open-records-agent")

In [139]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [140]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [116]:
agent_executor.invoke({"input": "what are the definitions for the open records statutes?", "chat_history": []})



> Entering new AgentExecutor chain...

Invoking: `lookup_statute_text` with `{'query': '61.870'}`


KRS 61.870: Definitions for KRS 61.870 to 61.884.
As used in KRS 61.870 to 61.884, unless the context requires otherwise:
	(1) “Public agency” means:
		(a) Every state or local government officer;
		(b) Every state or local government department, division, bureau, board, commission, and authority;
		(c) Every state or local legislative board, commission, committee, and officer;
		(d) Every county and city governing body, council, school district board, special district board, and municipal corporation;
		(e) Every state or local court or judicial agency;
		(f) Every state or local government agency, including the policy-making board of an institution of education, created by or pursuant to state or local statute, executive order, ordinance, resolution, or other legislative act;
		(g) Any body created by state or local authority in any branch of government;
		(h) Any body which, within 

{'input': 'what are the definitions for the open records statutes?',
 'chat_history': [],
 'output': 'Here are the definitions for the open records statutes in Kentucky:\n\n1. **KRS 61.870**: Defines "public agency" and "public record," including the types of entities and records covered by the law. It also defines "software," "commercial purpose," "official custodian," "custodian," "media," "mechanical processing," "booking photograph and photographic record of inmate," and "resident of the Commonwealth."\n\n2. **KRS 61.871**: Establishes the policy that free and open examination of public records is in the public interest and that exceptions to this policy should be strictly construed.\n\n3. **KRS 61.8715**: Recognizes the essential relationship between the intent of the open records law and the management of public records, as well as the coordination of strategic planning for computerized information systems in state government.\n\n4. **KRS 61.872**: Outlines the right of Kentucky 

In [141]:
agent_executor.invoke({"input": "Are police body cam and dash cam footage open records?", "chat_history": []})



> Entering new AgentExecutor chain...

Invoking: `lookup_statute_text` with `{'query': '61.878'}`


KRS 61.878: Certain public records exempted from inspection except on order of court — Restriction of state employees to inspect personnel files prohibited.
(1) The following public records are excluded from the application of KRS 61.870 to 61.884 and shall be subject to inspection only upon order of a court of competent jurisdiction, except that no court shall authorize the inspection by any party of any materials pertaining to civil litigation beyond that which is provided by the Rules of Civil Procedure governing pretrial discovery:
	(a) Public records containing information of a personal nature where the public disclosure thereof would constitute a clearly unwarranted invasion of personal privacy;
	(b) Records confidentially disclosed to an agency and compiled and maintained for scientific research. This exemption shall not, however, apply to records the disclosure or publication o

{'input': 'Are police body cam and dash cam footage open records?',
 'chat_history': [],
 'output': 'Police body cam and dash cam footage can be considered open records under the Kentucky Open Records Act, but there are specific conditions and exemptions that may apply. According to KRS 61.878, certain public records are exempted from inspection except on order of a court. This includes records of law enforcement agencies that were compiled in the process of detecting and investigating statutory or regulatory violations if the disclosure of the information would harm the agency by revealing the identity of informants not otherwise known or by premature release of information to be used in a prospective law enforcement action or administrative adjudication. However, unless exempted by other provisions, public records exempted under this provision shall be open after enforcement action is completed or a decision is made to take no action.\n\nCase law provides further clarification. In th

In [142]:
agent_executor.invoke({"input": "Are prisoner and inmate records in public institutions open records?", "chat_history": []})



> Entering new AgentExecutor chain...

Invoking: `lookup_statute_text` with `{'query': '61.878'}`


KRS 61.878: Certain public records exempted from inspection except on order of court — Restriction of state employees to inspect personnel files prohibited.
(1) The following public records are excluded from the application of KRS 61.870 to 61.884 and shall be subject to inspection only upon order of a court of competent jurisdiction, except that no court shall authorize the inspection by any party of any materials pertaining to civil litigation beyond that which is provided by the Rules of Civil Procedure governing pretrial discovery:
	(a) Public records containing information of a personal nature where the public disclosure thereof would constitute a clearly unwarranted invasion of personal privacy;
	(b) Records confidentially disclosed to an agency and compiled and maintained for scientific research. This exemption shall not, however, apply to records the disclosure or publication o

{'input': 'Are prisoner and inmate records in public institutions open records?',
 'chat_history': [],
 'output': 'Prisoner and inmate records in public institutions are generally considered open records under the Kentucky Open Records Act, subject to certain limitations and exemptions as outlined in KRS 61.878 and relevant case law and Attorney General opinions.\n\n1. **General Accessibility**: The Kentucky Open Records Act does not identify any class or type of persons, including prisoners, who are held to a more stringent standard when submitting open records requests. This presumption in favor of broad availability of public records is even stronger when a person, like an inmate, seeks access to public records pertaining to themselves (Commonwealth v. Chestnut, 2008).\n\n2. **Right to Inspect Public Records**: A prison inmate has the same right to inspect public records as any other person. The identity of the requester is therefore irrelevant. However, an agency is not required to

In [143]:
agent_executor.invoke({"input": "Can I access aggregated records related to school discipline, suspensions, and expulsions?", "chat_history": []})



> Entering new AgentExecutor chain...

Invoking: `lookup_exception_text` with `{'query': '61.878(1)(a)'}`


Exception 61.878(1)(a): Public records containing information of a personal nature where the public disclosure thereof would constitute a clearly unwarranted invasion of personal privacy;

There are some caveats to any exemption from 61.878:
61.878(2) No exemption in this section shall be construed to prohibit disclosure of statistical information not descriptive of any readily identifiable person.
61.878(3) No exemption in this section shall be construed to deny, abridge, or impede the right of a public agency employee, including university employees, an applicant for employment, or an eligible on a register to inspect and to copy any record including preliminary and other supporting documentation that relates to him. The records shall include, but not be limited to, work plans, job performance, demotions, evaluations, promotions, compensation, classification, reallocation, tr

{'input': 'Can I access aggregated records related to school discipline, suspensions, and expulsions?',
 'chat_history': [],
 'output': 'Yes, you can access aggregated records related to school discipline, suspensions, and expulsions under the Kentucky Open Records Act, with certain conditions to ensure privacy.\n\nThe key points to consider are:\n\n1. **Personal Privacy Exemption**: The exemption under KRS 61.878(1)(a) protects records containing personal information where disclosure would be an unwarranted invasion of privacy. However, this does not prohibit the disclosure of statistical information that is not descriptive of any readily identifiable person (KRS 61.878(2)).\n\n2. **Case Law and Attorney General Opinions**:\n   - A county school system was required to release statistical information relating to student disciplinary hearings without revealing personal characteristics of the student that would lead to identification, such as name or age (Hardin County Schs v. Foster, 20

In [144]:
agent_executor.invoke({"input": "If the Fish and Wildlife Department conducts all of their business on private email addresses, are those emails open records?", "chat_history": []})



> Entering new AgentExecutor chain...

Invoking: `find_case_law_annotations` with `{'query': 'private email use for public business'}`


While the interests reckoned on the privacy side of the balance generally do not dissipate, and in some instances even grow stronger, with the passage of time, it is no less true that the public’s interest in knowing what the government is up to includes a strong historical interest in knowing what the government was up to; the passage of time, therefore, while a factor relevant to the balancing of interests required by the privacy exemption, will seldom be dispositive in-and-of itself. Lawson v. Office of the AG, 415 S.W.3d 59, 2013 Ky. LEXIS 640 (Ky. 2013).
Category: Construction
Statute Number: 61.878
Type: Annotation


Ky. Rev. Stat. Ann. § 61.878(1)(a) did not permit the withholding of the entire internal affairs files, but disclosure of information that was personal in nature, such as social security numbers, was prohibited and constituted an 

{'input': 'If the Fish and Wildlife Department conducts all of their business on private email addresses, are those emails open records?',
 'chat_history': [],
 'output': 'According to an Attorney General Opinion (OAG 03-ORD-05), electronic mail generated by public agency officials or employees is considered a public record as defined in KRS 61.870(2) and is therefore subject to the Open Records Act. This means that it is the responsibility of the records custodian, not the email account holder, to locate and retrieve these records and to make the determination as to which records are exempt and which must be disclosed. This applies regardless of whether the emails are sent or received through private email addresses or official government email accounts, as long as they pertain to public business conducted by the agency.\n\nTherefore, if the Fish and Wildlife Department conducts all of their business on private email addresses, those emails would still be considered open records under